Geçenlerde kıyafet beden analizi için kullandığım datayı bu sefer beden ölçüsü tahmini için kullandım.

Amaç: Kilo ve boy bilgisi girildiğinde size bilgisine ulaşmak

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
data=pd.read_csv("../input/clothessizeprediction/final_test.csv")

In [ ]:
data.head()

In [ ]:
data.groupby("size")[["weight","height"]].mean()

In [ ]:
data.isnull().sum()

In [ ]:
data["height"].fillna(data["height"].mean(),inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data[data["age"]==0].head()

In [ ]:
data[data["size"]=="S"].describe() 

In [ ]:
data[(data["weight"]<40) & (data["size"]!="S") & (data["size"]!="XXS")]

In [ ]:
data["age_weight_height"]=[str(i[1])+"_"+str(i[0])+"_"+str(i[2]) for i in data.values]
data.head()

In [ ]:
b=data["age_weight_height"].value_counts().index
b

In [ ]:
def customer_check(i,a,c):
    data.loc[data["age_weight_height"]==a,"customer_id"]=c

In [ ]:
c=0
for i in range(14149):
    if b[i] in data["age_weight_height"].values:
        c+=1
        customer_check(i,b[i],c)

In [ ]:
data["customer_id"]=[int(i) for i in data["customer_id"]]
data.head()

In [ ]:
data["age_weight_height"].nunique() #aynı nitelikte müşteriler var sayı 119734'ten 14149'a inmiş çünkü

### Yalnızca tek parça kıyafet alan kişilerin analizini yapalım. 

In [ ]:
a=data.groupby("customer_id")[["size"]].count()

In [ ]:
a.loc[5,"size"]

In [ ]:
#burada sadece tek bir parça ürün alan kişinin feature sütununa 1, daha fazla alan kişininkine de 0 atıyorum.
for i in range(1,14150):
    if a.loc[i,"size"]==1:
        a.loc[i,"feature"]=1
    elif a.loc[i,"size"]>1:
        a.loc[i,"feature"]=0

In [ ]:
dnew=pd.merge(data,a, on="customer_id")
dnew.head()

In [ ]:
dnew["feature"]=dnew["feature"].astype(int)

In [ ]:
df=dnew[dnew["feature"]==1]

In [ ]:
df.describe()

In [ ]:
df["customer_id"].nunique()

In [ ]:
dta=df.groupby("size_x")[["weight","height"]].mean()
dta

In [ ]:
#Analiz öncesi ortalamalar
data.groupby("size")[["weight","height"]].mean()


*************************************** SİZE FONKSİYONU ****************************************

In [ ]:
dta

In [ ]:
dta["rate"]=dta["weight"]/dta["height"]

In [ ]:
dta

In [ ]:
dta=dta.sort_values(by="rate")

In [ ]:
dta

rate hesabıyla çok anlamlı bir veri elde ettim.

In [ ]:
dta.iloc[1,2]

Veride xxs ve s var ve aralarındaki oran çok yüksek değil o yüzden xxs'i xs olarak düşündüm.

In [ ]:
def size(w,h):
    if h<3:
        h=h*100
    a=w/h
    if a < dta.iloc[1,2]:
        c="XS"
    elif a < dta.iloc[2,2]:
        c="S"
    elif a < dta.iloc[3,2]:
        c="M"
    elif a < dta.iloc[4,2]:
        c="L"
    elif a < dta.iloc[5,2]:
        c="XL"
    elif a < dta.iloc[6,2]:
        c="XXL"
    elif a >= dta.iloc[6,2]:
        c="XXXL" 
    return f"Ölçüleriniz {c} bedene uymaktadır." 

size fonksiyonuna önce kilo sonra ise boy ölçüsü girilir(m veya cm olarak girilebilir.)

In [ ]:
size(75,170)

In [ ]:
size(90,1.70)

In [ ]:
size(48,170)

In [ ]:
size(68,1.60)

In [ ]:
size(55,1.50)

In [ ]:
size(60,150)